# Import HTML texts into a PostgreSQL database and extract new data

* scrape the source website and get the textual data
* transform the data into XML and extract the text as such




## Site original


For the preparation and exploration of import list, see _explore_ notebook

Licence : 

[Copyright Information](https://mathshistory.st-andrews.ac.uk/Miscellaneous/copyright/) (Original website) -> [Creative Commons License 4.0 BY SA](http://creativecommons.org/licenses/by-sa/4.0/)


In [1]:
### Importation de modules
import requests
import psycopg2
import lxml.html
from lxml import etree
from importlib import reload
import sqlite3 as sql

In [2]:
import urllib3
### Avoid disturbing messages while scraping
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
import settings as st

## Explore mathematician's list

In [3]:
# Mathematicians from the year 1000 and onward 
f = "html_files/chrono_biogr_index_1000_and_later.html"
a = etree.parse(f)
a

In [4]:
### Test the syntax to extract the URLs
a.xpath('//li/a/@href')[0:2]

['https://mathshistory.st-andrews.ac.uk/Biographies/Al-Nasawi/',
 'https://mathshistory.st-andrews.ac.uk/Biographies/Jia_Xian/']

In [5]:
### Extract all URLs
l_url = a.xpath('//li/a/@href')
print(len(l_url))
### syntax for relative URLs, not needed here
# restructured_url = ['https://mathshistory.st-andrews.ac.uk/' + url.replace('../../', '') for url in l_url]
urls = [url for url in l_url]
len(urls)

3010


3010

In [6]:
urls[:2]

['https://mathshistory.st-andrews.ac.uk/Biographies/Al-Nasawi/',
 'https://mathshistory.st-andrews.ac.uk/Biographies/Jia_Xian/']

In [14]:
### Prepare the whole list of persons
t = a.xpath('//li') #[0:2]

In [15]:
### Extract birth/death year, name, URL 
l_date_nom_url = [[e.xpath('./text()')[0].replace('(', '').replace(')', '').replace(' - ', '-').strip(), \
                   e.xpath('./a/text()')[0], \
                   e.xpath('./a/@href')[0]] \
                   for e in t]

In [16]:
print(len(l_date_nom_url))
l_date_nom_url[10:13], l_date_nom_url[-3:]

3010


([['1089-1173',
   'Hemchandra',
   'https://mathshistory.st-andrews.ac.uk/Biographies/Hemchandra/'],
  ['1092-1167',
   'Rabbi Ben Ezra',
   'https://mathshistory.st-andrews.ac.uk/Biographies/Ezra/'],
  ['1100-1160',
   'Jabir ibn Aflah',
   'https://mathshistory.st-andrews.ac.uk/Biographies/Jabir_ibn_Aflah/']],
 [['1979-',
   'Artur Ávila',
   'https://mathshistory.st-andrews.ac.uk/Biographies/Avila/'],
  ['1984-',
   'Maryna Viazovska',
   'https://mathshistory.st-andrews.ac.uk/Biographies/Viazovska/'],
  ['1984-',
   'Qëndrim Gashi',
   'https://mathshistory.st-andrews.ac.uk/Biographies/Gashi/']])

In [ ]:
### POur ne pas demander le password — ou variable d'environnement améliorer
# https://www.delftstack.com/howto/python/import-python-file-from-path/#import-a-module-with-a-full-file-path-in-python-35from importlib.machinery import SourceFileLoader

MODULE_PATH = "c:\\Users\\Rexjohn\\Folder-1\\addmodule.py"
MODULE_NAME = "addmodule"

modulevar = SourceFileLoader(MODULE_NAME, MODULE_PATH).load_module()

modulevar.printingstatement()


## PostgreSQL

## Documentation

* https://www.psycopg.org/docs/usage.html
* https://towardsdatascience.com/python-and-postgresql-how-to-access-a-postgresql-database-like-a-data-scientist-b5a9c5a0ea43
  * noter toutefois que la connection directe avec psychopg2 et pandas ne marche plus
  
__NB__ 
* soit on passe par du SQL pur, et alors on peut utiliser _psycopg2_ comme on utilise sqlite3
* soit on utilise Pandas, mais là il faut un objet SQLAlchemy pour accéder à la base de données  
  
  
  

In [7]:
conn = psycopg2.connect(host="localhost", port = 5432, database="espace_intellectuel", 
                        user="postgres", password=st.dbw)

In [8]:
conn

<connection object at 0x7fd051624f40; dsn: 'user=postgres password=xxx dbname=espace_intellectuel host=localhost port=5432', closed: 0>

In [178]:
create_table_sql = """CREATE TABLE IF NOT EXISTS astronomers.mathshistory (
                                        pk_mathshistory serial PRIMARY KEY,
                                        name varchar NOT NULL,
                                        dates varchar,
                                        url varchar,
                                        short_name varchar,
                                        page_html xml,
                                        page_html_txt text,
                                        notes text, 
                                        ts_creation timestamp);"""

In [58]:
### Créer la table dans la base de données
with conn.cursor() as curs:
    # curs.execute(create_table_sql)
    # conn.commit()

### Exploration page

In [7]:
### view-source:https://mathshistory.st-andrews.ac.uk/Biographies/Roberval/
url = 'https://mathshistory.st-andrews.ac.uk/Biographies/Roberval/'

In [8]:
html = requests.get(url, verify=False, )

In [ ]:
print(html.content.decode())

In [22]:
l_date_nom_url[173]

['1602-1675',
 'Gilles Roberval',
 'https://mathshistory.st-andrews.ac.uk/Biographies/Roberval/']

## Data import

In [ ]:
### Production query

errors_list = []
parser = etree.XMLParser(remove_blank_text=True)
# for i in l_date_nom_url[10:12]: [:61] [173:174]
for i in l_date_nom_url:
    url = i[2]

    html = requests.get(url, verify=False, )
    ### Use document_fromstring to produce valid XML
    ## https://lxml.de/lxmlhtml.html#parsing-html
    b = lxml.html.document_fromstring(html.content.decode())
    # b = lxml.html.document_fromstring(html.text)
    divs_row = b.xpath('//div[@class="row"]')    
    
    content = '\n'.join([etree.tostring(e, encoding='utf8').decode() for e in divs_row])
    txt = '<root>' + content + '</root>' 
    #print(txt)
    value = ((i[1],i[0],i[2],txt))
    
    with conn.cursor() as curs:
        try:   
            ### Décommenter pour nouvelle exécution
            # curs.execute("""INSERT INTO astronomers.mathshistory (name, dates, url, page_html_txt)
              #  VALUES (%s, %s, %s, %s);""",
                value)
            # conn.commit()
        except Exception as e:
            errors_list.append([i[2], e])
            print(e)

            

In [10]:
def sql_explore(q):
    errors_list = []
    elem = []
    with conn.cursor() as curs:
        try:   
            ### Décommenter pour nouvelle exécution
            curs.execute(q)
            elem = curs.fetchall()
            conn.commit()
        except Exception as e:
            errors_list.append([i[2], e])
            print(e)
    return [elem, errors_list]        

In [11]:
q1 = """
SELECT COUNT(*), min(ts_creation), max(ts_creation)
FROM astronomers.mathshistory;
"""

In [38]:
q2 = """
SELECT pk_mathshistory, "name", dates, url FROM astronomers.mathshistory WHERE xml_is_well_formed(page_html_txt) IS FALSE ;
"""

In [17]:
### Muliple cards per URL ?
# No one, empty set
q3 = """
select url, array_agg(pk_mathshistory) 
from espace_intellectuel.astronomers.mathshistory m
group by url 
having count(*) > 1;
"""

In [19]:
### Muliple cards per name ?
## Result:
# Lines count: 4, errors count: 0, 
# First lines: [('Gustav von Escherich', [894, 895]), ('John Wilson', [875, 396]), ('Lloyd Williams', [1537, 2536]), ('William Thomson', [700, 992])]
# Deleted one line: https://mathshistory.st-andrews.ac.uk/Biographies/Esterich/ — see below

q4 = """
select "name", array_agg(pk_mathshistory) 
from espace_intellectuel.astronomers.mathshistory m
group by "name" 
having count(*) > 1;
"""

In [21]:
result = sql_explore(q4)
print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

Lines count: 4, errors count: 0, 
First lines: [('Gustav von Escherich', [894, 895]), ('John Wilson', [875, 396]), ('Lloyd Williams', [1537, 2536]), ('William Thomson', [700, 992])]


## Create valid XML in a dedicated field


I had to manually clean up some records because of unclean UTF-8 characters

Relevant query to find the errors:
```SQL
SELECT page_html_txt, * FROM astronomers.mathshistory WHERE xml_is_well_formed(page_html_txt) IS FALSE LIMIT 50;
```



In [22]:
qu = """UPDATE astronomers.mathshistory SET page_html = page_html_txt::XML
WHERE xml_is_well_formed(page_html_txt) IS TRUE ;"""

In [23]:
with conn.cursor() as curs:
    try:   
        ### Décommenter pour nouvelle exécution
        curs.execute(qu)
        conn.commit()
    except Exception as e:
        print(e)

In [24]:
q = """
select * from astronomers.mathshistory m where length(page_html::text) = 0;
"""

In [25]:
result = sql_explore(q)
print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

Lines count: 0, errors count: 0, 
First lines: []


## Additional information


For some persons there is additional information on other parts of the website

For instance:
* https://mathshistory.st-andrews.ac.uk/Biographies/Esterich/ (deleted from database, pk_mathshistory = 894)
* https://mathshistory.st-andrews.ac.uk/Biographies/Escherich/ (pk_mathshistory = 894)
point to:
* https://mathshistory.st-andrews.ac.uk/Societies/Austrian/#Escherich


To be discussed: extract also this information that is, in this case, related to __organisations__ these persons belong to

E.G the [Austrian Mathematical Society](https://mathshistory.st-andrews.ac.uk/Societies/Austrian/) (1903-)

### Structure of website HTML

```HTML
<div class="row">
  <div class="col-md-12">
    <h3>Biography</h3>
    <span class="markup">This short biography is in the <strong>Societies</strong> folder.
<br/>

<br/>
To see it press <a href="../../Societies/Austrian/#Escherich">THIS LINK</a>.</span>
  </div>
</div>
```


How many there are in the website ?

In [30]:
q = """
with tw1 as (
SELECT pk_mathshistory, name, substr(dates, 1, 4)::INTEGER birth_year,
array_to_string(xpath('//div[./h3/text()="Biography"]//a[contains(text(), "THIS LINK")]/@href[contains(., "ocieties")]', page_html), ',') as links
FROM astronomers.mathshistory)
select *
from tw1
where length(links) > 5;
"""

In [31]:
result = sql_explore(q)
print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:5]}')

Lines count: 40, errors count: 0, 
First lines: [(115, 'Henry Savile', 1549, '../../Societies/Savilian,../../Societies/Savilian'), (231, 'Valentin Heins', 1637, '../../Societies/Hamburg/#Heins'), (241, 'Heinrich Meissner', 1644, '../../Societies/Hamburg/#Meissner'), (894, 'Gustav von Escherich', 1849, '../../Societies/Austrian/#Escherich'), (895, 'Gustav von Escherich', 1849, '../../Societies/Austrian/#Escherich')]


## Explore XML texts





In [34]:
### Birth-Places
q3 = """
SELECT name, dates, 
(xpath('//dt[text()="Born"]/following-sibling::dd/a/text()', page_html))[1]::text as lieu_naiss,
(xpath('//dt[text()="Born"]/following-sibling::dd/a/@href', page_html))[1]::text as url_lieu_naiss
FROM astronomers.mathshistory
--WHERE substr(dates, 1, 4)::INTEGER BETWEEN 1601 and 1650;
"""

In [35]:
result = sql_explore(q3)
print(f'Lines count: {len(result[0])}, errors count: {len(result[1])}, \nFirst lines: {result[0][:3]}')

Lines count: 3010, errors count: 0, 
First lines: [('John Colson', '1680-1760', ' Lichfield, Staffordshire, England ', '../../Map/#Lichfield'), ('Scott Lang', '1850-1926', ' Edinburgh, Scotland ', '../../Map/#Edinburgh'), ('J A Third', '1865-1948', ' St Nicholas, Aberdeen, Scotland ', '../../Map/#Aberdeen')]


In [1]:
q1 = """
with tw1 as (
SELECT pk_mathshistory, name, substr(dates, 1, 4)::INTEGER birth_year, dates, 
unnest(xpath('//dt[text()="Summary"]/following-sibling::dd/*/descendant-or-self::*/text()', page_html))::text as summary,
unnest(xpath('//h3[text()="Biography"]/following-sibling::span/descendant-or-self::*[not(@class="translation nonoscript non-italic")]/text()', page_html))::text as biography,
(xpath('//dt[text()="Born"]/following-sibling::dd/a/text()', page_html))[1]::text as lieu_naiss,
(xpath('//dt[text()="Born"]/following-sibling::dd/a/@href', page_html))[1]::text as url_lieu_naiss
FROM astronomers.mathshistory 
WHERE substr(dates, 1, 4)::INTEGER BETWEEN 1563 and 1575
)
SELECT pk_mathshistory, name, birth_year, dates, string_agg(summary, '') as summary, 
string_agg(regexp_replace(biography, '\n', ' '), '') as biography, lieu_naiss, url_lieu_naiss
FROM tw1
GROUP BY pk_mathshistory, name, birth_year, dates, lieu_naiss, url_lieu_naiss
ORDER BY birth_year;
"""

In [ ]:
errors_list = []
elem = []
with conn.cursor() as curs:
    try:   
        ### Décommenter pour nouvelle exécution
        curs.execute(q1)
        elem = curs.fetchall()
        ## conn.commit()
    except Exception as e:
        print(e)
conn.commit()        

In [ ]:
conn.close()

In [ ]:
errors_list

In [ ]:
len(elem), elem[1]

In [17]:
t = elem[10][5].replace('\n', '').replace("\'", '')[:100]
t

'Giovanni Alfonso Borellis father was Miguel Alonso and his mother was Laura Porrello. The first obvious question must be to ask about the name Borelli. It is a somewhat complicated story, but the subject of this biography was baptised Giovanni Francesco Antonio Alonso. His father, Miguel Alonso, was a Spanish soldier stationed in Naples as part of the Spanish garrison. The city had been captured by the Spanish in 1503 and remained under Spanish control for over 200 years. The Spanish soldiers were stationed in the three forts of Naples, the Castel SantElmo, the Castel dellOvo, and the Castel Nuovo where Miguel Alonso was when Giovanni was born. His mother, Laura Porrello, was a local girl and Giovanni was the eldest of his parents six children; five boys, Giovanni, Domenico Onofrio, Salvatore, Filippo, Diego and one girl, Caterina. We will see below why, later in his life, Giovanni wanted to change his name. Let us just point out at this stage that he dropped his fathers name Alonso a

In [67]:
conn.close()

In [58]:
db_path = 'data/mathshistory_web.sqlite'

In [61]:
conn = sql.connect(db_path)

In [155]:
create_table_sql = """ CREATE TABLE IF NOT EXISTS mathematiciens (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        dates text,
                                        url text,
                                        short_name text,
                                        page_html text
                                    ); """

In [62]:
c = conn.cursor()
### Uncomment if needed
### c.execute(create_table_sql)

In [161]:
conn = sqlite3.connect(db_path)
c = conn.cursor()

In [162]:
# c.executemany('INSERT INTO mathematiciens(name, dates, url, short_name, page_html) VALUES (?,?,?,?,?)', insert_list)

In [ ]:
conn.commit()

In [63]:
c = conn.cursor()

In [64]:
c.execute('SELECT * FROM mathematiciens WHERE CAST(substr(dates, 1, 4) as INTEGER) BETWEEN 1601 and 1650  ')

In [65]:
result = []
result = c.fetchall()
conn.close()

In [66]:
for r in result[:5]:
    print(r[1],r[2])

Florimond de Beaune 1601-1652
Pierre Fermat 1601-1665
Gilles Roberval 1602-1675
Jacques de Billy 1602-1679
John Greaves 1602-1652


In [ ]:
conn.close()